<h1 style="text-align: center;">Caso de Estudio - Entrega Final</h1>

<h4>Grupo: Los Analíticos</h4> 
<ul>
<li>Sofia Isabella Endara Chitiva</li>
<li>Nestor Andrés David Tabares</li>
<li>Juan David Lasso</li>
<li>Nicolas Joel Caceres Parra</li>
</ul>

El enfoque de este cuadernillo de python es aplicar aprendizaje por refuerzo (RL) basado en modelos de Markov, para desarrollar una política de inversión óptima en divisas, considerando las dinámicas previamente identificadas. La meta es maximizar las ganancias a lo largo de un horizonte de planificación finito de tres días, considerando posibles acciones de mantener o cambiar entre divisas.

## Librerías

In [1]:
import pandas as pd #Para manejo de datos en DataFrames y funciónes de analítica de datos
import numpy as np #Para manejo de datos numéricos y arreglos
import matplotlib.pyplot as plt #Para la visualización de datos
import seaborn as sns #Para la visualización de datos
import scipy.stats as stats #Para realizar pruebas de hipótesis y estadiísticas
import warnings #Para evitar promptes de advertencia
import scipy.stats as stats # Para realizar ajustes de normalidad
import rp4solver310 as rp4 # Librería para el uso de modelos de MDP
from rp4solver310 import atico # Importar la clase atico para desarrollar el modelo
warnings.filterwarnings('ignore') # evita warnings innecesarios al ejecutar código

RP4 Solver Copyright(c) 2022
Version 6.2 - Beta version
Developed by Alfonso T. Sarmiento



# Extracción y Limpieza de Datos

In [2]:
link_database = 'https://raw.githubusercontent.com/NicolasCacer/bases-de-datos/main/1.2.TCM_Serie%20para%20un%20rango%20de%20fechas%20dado.csv'

In [3]:
def load_data(url):
    df = pd.read_csv(url)
    df.columns = ['Fecha', ' Año', 'Nombre_Moneda', 'Continente', 'Cambio', 'Tipo_tasa', 'TRM', 'Id_Moneda']
    weekends = df['Fecha'].unique().tolist()[2::7]+df['Fecha'].unique().tolist()[3::7]
    keep = set(df['Fecha'].unique().tolist())
    dates = keep.difference(weekends)
    dates =list(dates)
    dates.sort()
    df = df[df['Fecha'].isin(dates)]
    COP_Med_Data = df.query("Cambio == 'Pesos colombianos por cada moneda' and Tipo_tasa == 'Media'")
    COP_Med_Data['TRM'] = COP_Med_Data['TRM'].str.replace(',','.').astype(float)
    data_original = pd.DataFrame(columns=df['Id_Moneda'].unique().tolist(), index=df['Fecha'].unique())
    for i in data_original.index:
        row = list(COP_Med_Data.query("Fecha == @i")['TRM'])
        data_original.loc[i] = row
    data_original = data_original.astype(float)
    data_original.head()
    data = data_original.copy()
    data.reset_index(drop=True, inplace=True)
    return data

data = load_data(url=link_database)

El código carga datos de un **`CSV`** y renombra las columnas, luego identifica y elimina las **`fechas`** que caen en fines de semana, filtra los datos para incluir solo aquellos donde la tasa de cambio es en pesos colombianos y de tipo **`Media`**, y convierte la columna **`TRM`** de texto a valores numéricos. Posteriormente, organiza los datos en un nuevo DataFrame con fechas como índices y los Id_Moneda como columnas, rellenando las celdas con los valores de TRM correspondientes y reseteando el índice para preparar el DataFrame para su uso posterior.

## Construcción de Modelo de Markov

In [4]:
Estados = ['B','S']
def final_states(D=data, currency="JPY", n_per=20,only_states=True,rolling_std=False):  # la ventana de tiempo para la desviación móvil es 20 días
    currency = currency.upper()
    D = pd.DataFrame(data[currency])
    D['Mob_std'] = D[currency].rolling(n_per).std()
    D['Cambio_%'] = D[currency].pct_change()
    D['Estado'] = np.where(D['Cambio_%'] < 0, Estados[0], Estados[1])
    if rolling_std:
        D['sigma'] = D['Cambio_%'].rolling(n_per).std()
    D = D[n_per:]
    if only_states:
        D = D.drop(columns=[currency, 'Mob_std', 'Cambio_%'])
    D.reset_index(drop=True, inplace=True)
    return D

La función `final_states` toma un DataFrame con datos de tipos de cambio , una moneda específica, y otros parámetros opcionales, y devuelve un DataFrame con información sobre los estados de cambio de la moneda `B` para Bajada y `S` para subida en base al porcentaje de cambio diario y la desviación movil.

In [5]:
def markoviana(currency = "JPY", n_periods = 20, prints=True):

    Data = final_states(currency= currency, n_per= n_periods)
    Data.columns = ["x_t"]
    Data["x_t+1"] = Data["x_t"].shift(-1)
    Data["x_t+2"] = Data["x_t+1"].shift(-1)
    freq_obs = pd.DataFrame(columns=("Anterior", "Actual"))
    for i in Estados:
        for j in Estados:
            freq_obs.loc[len(freq_obs)] = [i,j]
    freq_obs = pd.concat([freq_obs,pd.DataFrame(np.zeros((freq_obs.shape[0],len(Estados)+1)),columns=Estados+['Total'])], axis=1)
    for i in range(len(Data)-2):
        secuencia = Data.loc[i].tolist()
        freq_obs.loc[(freq_obs['Anterior'] == secuencia[0]) & (freq_obs['Actual'] == secuencia[1]),secuencia[2]] += 1
    freq_obs['Total'] = freq_obs[Estados].sum(axis=1)
    freq_esp = freq_obs.copy()
    freq_esp.iloc[:,2:] = 0
    for i in range(len(Data)-2):
        secuencia = Data.loc[i].tolist()
        freq_esp.loc[freq_esp['Actual'] == secuencia[1],secuencia[2]] += 1
    freq_esp['Total'] = freq_esp[Estados].sum(axis=1)
    porcentaje_menor_5 = (freq_obs[Estados] < 5).mean() * 100
    hay_ceros = (freq_obs["Total"] == 0).any()
    porcentaje_menor_5 = (freq_obs[Estados] < 5).mean() * 100
    hay_ceros = (freq_obs["Total"] == 0).any()
    if (porcentaje_menor_5.mean() > 25 or hay_ceros) and prints:
        print("Se necesito agrupar dado que los datos menores a 5 represetan más del 25% del total o hay 0 en la columna total")
        print("---"*100)
        freq_obs = freq_obs.groupby("Anterior").sum().reset_index()
        freq_esp = freq_esp.groupby("Anterior").sum().reset_index()
    prob_obs = freq_obs[Estados].div(freq_obs["Total"], axis=0)
    prob_esp = freq_esp[Estados].div(freq_esp["Total"], axis=0)
    resultado = ((prob_obs - prob_esp) ** 2 / prob_esp).sum(axis=1) * freq_obs["Total"]
    chi_2 = (((prob_obs - prob_esp) ** 2 / prob_esp).sum(axis=1) * freq_obs["Total"]).sum()
    deg_free = (prob_esp.shape[1]-1)*(prob_esp.shape[0]-1)
    alpha = 0.05
    chi2_inv = stats.chi2.ppf(1-alpha, deg_free)
    if chi_2 < chi2_inv:
        Markov = "No se rechaza Ho"
    else:
        Markov = "Se rechaza Ho"
    freq_Markov = pd.DataFrame(0,columns= Estados+["Total"], index=Estados)
    for i in range(len(Data)-1):
        secuencia = Data.loc[i].tolist()[:2]
        freq_Markov.loc[secuencia[0],secuencia[1]]+= 1
    freq_Markov["Total"] = freq_Markov.sum(axis=1)
    
    prob_Markov = freq_Markov[Estados].div(freq_Markov["Total"], axis=0)
    

    if prints:
        print(f"-> Hipotesis markoviana\n {Markov}")
        print("-----"*10)
        print(f"-> Matriz de transición")
        print(prob_Markov.round(4))
        
    return prob_Markov

La función `markoviana` analiza una serie temporal de estados de cambio de una moneda y realiza una prueba para verificar si la secuencia sigue una propiedad markoviana; es decir, si el estado futuro depende solo del estado actual y no de los anteriores.

In [6]:
markoviana(currency = "JPY", n_periods = 20, prints=False).stack().index.tolist()

[('B', 'B'), ('B', 'S'), ('S', 'B'), ('S', 'S')]

# Procesos de Decisión Markovianos

## Modelado del proceso de decisión

## Estructuración del modelo MDP

Un modelo de decisión Markoviano consiste en un algoritmo que busca maximizar o minimizar la ganancia o el costo de las decisiones tomadas por un agente en medio de su entorno, con el conocimiento del nivel de incertidumbre que posee cada uno de los posibles estados. Consecuentemente, se busca generar un modelo que llegue a una política de inversión óptima para maximizar la rentabilidad obtenida durante 3 días al invertir dinero entre dos divisas diferentes. Para esto, se parte de la verificación de que las monedas cumplan propiedades markovianas y de homogeneidad, lo cual se puede validar con funciones markovianas realizadas.

A partir de ese punto, se construye el modelo mediante el modelamiento SATR, el cual consiste en la formulación de los estados del sistema, las acciones posibles en cada estado y las recompensas asociadas a cada acción; adicionalmente, se debe plantear un vector de estado, estado de partida, conjunto de acciones, conjunto de eventos, ecuaciones de transición, restricciones, probabilidades de transición, contribución de la acción, contribución del evento, función de calidad, función de valor óptimo y condición de contorno. Todos estos elementos son los que se desarrollan a continuación.

### Vectores de Estado

Se define el vector de estado como:

$$ S = (s_1, s_2, s_3, s_4, s_5, s_6) $$

Donde:

- \( $s_1$ \): Número de día en el cual se realiza una inversión.
- \( $s_2$ \): Divisa elegida para el día \( $s_1$ \).
- \( $s_3$ \): Estado del JPY en el día \( $s_1$ \).
- \( $s_4$ \): Estado del CHF en el día \( $s_1$ \).
- \( $s_5$ \): Precio de JPY en el día \( $s_1$ \).
- \( $s_6$ \): Precio de CHF en el día \( $s_1$ \).


### Estado Inicial

El estado inicial está dado por:

$$ (s_1, s_2, s_3, s_4, s_5, s_6) = (1, JPY, \text{Sube}, \text{Sube}, 100, 100) $$

Donde:

- La cuenta de los días inicia en la mañana de cada día a partir del día 1.
- De las monedas disponibles JPY, GBP, MXN, CHF, solo se utilizarán dos: JPY y CHF.
- En ambas monedas se inicia con un valor de 100 pesos colombianos.
- Las dos divisas manejarán un estado inicial de subida de precio.


### Restricciones del modelo

El horizonte de tiempo es finito, y está fijado a un máximo de 3 días de inversión:

$$ s_1 \leq 3 $$



### Conjunto de Acciones

El conjunto de acciones posibles es:

$$ AS = \{ \text{mantener divisa}, \text{cambiar divisa} \} \quad \forall s \in S $$



### Conjunto de Eventos

El conjunto de eventos posibles es:

$$
ES =
\begin{cases}
\text{Sube JPY, Sube CHF} & \forall a \in AS \\
\text{Sube JPY, Baja CHF} & \forall a \in AS \\
\text{Baja JPY, Baja CHF} & \forall a \in AS \\
\text{Baja JPY, Sube CHF} & \forall a \in AS
\end{cases}
$$

### Ecuaciones de Transición

1. Día siguiente:

    $$ s_{n1} = s_1 + 1, \forall s \in S $$

2. Divisa:

$$
s_{n2} =
\begin{cases}
\text{JPY}, & \text{si } (a = \text{mantener divisa} \land s_2 = \text{JPY}) \\
\text{JPY}, & \text{si } (a = \text{cambiar divisa} \land s_2 = \text{CHF}) \\
\text{CHF}, & \text{si } (a = \text{mantener divisa} \land s_2 = \text{CHF}) \\
\text{CHF}, & \text{si } (a = \text{cambiar divisa} \land s_2 = \text{JPY})
\end{cases}
$$

3. Estado JPY:

$$
s_{n3} =
\begin{cases}
\text{Sube}, & \text{si } e = (\text{Sube JPY}, \text{Sube CHF}) \vee (\text{Sube JPY}, \text{Baja CHF}) \\
\text{Baja}, & \text{si } e = (\text{Baja JPY}, \text{Sube CHF}) \vee (\text{Baja JPY}, \text{Baja CHF})
\end{cases}
$$

4. Estado CHF:

$$
s_{n4} =
\begin{cases}
\text{Sube}, & \text{si } e = (\text{Sube JPY}, \text{Sube CHF}) \vee (\text{Baja JPY}, \text{Sube CHF}) \\
\text{Baja}, & \text{si } e = (\text{Sube JPY}, \text{Baja CHF}) \vee (\text{Baja JPY}, \text{Baja CHF})
\end{cases}
$$

5. Precios JPY:


$$
s_{n5} =
\begin{cases}
s_5(1 + \sigma_i), & \text{si } e = (\text{Sube JPY, Sube CHF}) \vee (\text{Sube JPY, Baja CHF}) \\
s_5(1 - \sigma_i), & \text{si } e = (\text{Baja JPY, Sube CHF}) \vee (\text{Baja JPY, Baja CHF})
\end{cases}
$$

6. Precios CHF:

$$
s_{n6} =
\begin{cases}
s_6(1 + \sigma_i), & \text{si } e = (\text{Sube JPY, Sube CHF}) \vee (\text{Baja JPY, Sube CHF}) \\
s_6(1 - \sigma_i), & \text{si } e = (\text{Sube JPY, Baja CHF}) \vee (\text{Baja JPY, Baja CHF})
\end{cases}
$$


### Probabilidad de Transición

Para cada divisa, se define mediante una matriz \( 2 X 2 \):

$$
P_{JPY} =
\begin{pmatrix}
0.3941 & 0.6059 \\
0.4538 & 0.5462
\end{pmatrix}
$$

$$
P_{CHF} =
\begin{pmatrix}
0.3400 & 0.6600 \\
0.4238 & 0.5762
\end{pmatrix}
$$

Matriz conjunta:

$$
P_{JPY, CHF} =
\begin{pmatrix}
0.1340 & 0.2603 & 0.1675 & 0.2272 \\
0.2060 & 0.4000 & 0.2567 & 0.3493 \\
0.1543 & 0.2995 & 0.1922 & 0.2613 \\
0.1855 & 0.3605 & 0.2313 & 0.3143
\end{pmatrix}
$$

### Contribución de la Acción

- Al cambiar de divisa, se aplica una comisión del 1% sobre el precio de la divisa actual:

$$
C_a(a, s) =
\begin{cases}
-0.01 \cdot s_5 & \text{si } a = \text{cambiar divisa} \wedge s_2 = \text{JPY}, \\
-0.01 \cdot s_6 & \text{si } a = \text{cambiar divisa} \wedge s_2 = \text{CHF}, \\
0 & \text{en otro caso}.
\end{cases}
$$

### Contribución del Evento

La contribución del evento depende de la variación porcentual en el precio:

$$
C_e(a, s, e) = 
\begin{cases}
+s_5 \cdot \sigma_{\text{JPY}} & \text{si } (e = \text{Sube JPY, Sube CHF} \vee e = \text{Sube JPY, Baja CHF}) \wedge s_2 = \text{JPY}, \\
-s_5 \cdot \sigma_{\text{JPY}} & \text{si } (e = \text{Baja JPY, Sube CHF} \vee e = \text{Baja JPY, Baja CHF}) \wedge s_2 = \text{JPY}, \\
+s_6 \cdot \sigma_{\text{CHF}} & \text{si } (e = \text{Sube JPY, Sube CHF} \vee e = \text{Baja JPY, Sube CHF}) \wedge s_2 = \text{CHF}, \\
-s_6 \cdot \sigma_{\text{CHF}} & \text{si } (e = \text{Sube JPY, Baja CHF} \vee e = \text{Baja JPY, Baja CHF}) \wedge s_2 = \text{CHF}.
\end{cases}
$$


### Función de Calidad

$$Q_{s_a} = c_a + \sum_{i=1}^{m} p_i \left( c_{e_i} + V_{s_{n_i}} \right)$$

La ecuación anterior representa el valor de calidad \( $Q_{s_a}$ \) de tomar una acción \( a \) en un estado \( s \). A continuación, se explica cada uno de los términos de la ecuación:

- \( $Q_{s_a}$ \): Es el **valor de calidad** de tomar la acción \( a \) en el estado \( s \). Representa el beneficio total esperado de realizar la acción \( a \) en el estado \( s \) bajo la política actual del agente.
- \( $c_a$ \): Es el **costo o recompensa inmediata** asociada con la acción \( a \) en el estado \( s \). Este valor puede ser negativo (costo) o positivo (recompensa), y refleja el impacto inmediato de realizar la acción.
- \( $p_i$ \): Es la **probabilidad** de que ocurra el evento \( e_i \) después de tomar la acción \( a \) en el estado \( s \). Esta probabilidad describe la incertidumbre en las transiciones de estado.
- \( $c_{e_i}$ \): Es el **costo o recompensa** asociado con el evento \( e_i \), que puede ocurrir como resultado de tomar la acción \( a \) en el estado \( s \). Representa el impacto inmediato del evento \( e_i \) en el agente.
- \( $V_{s_{n_i}}$ \): Es el **valor del estado posterior** \( s_{n_i} \), que es el valor esperado del estado \($ s_{n_i}$ \) que se alcanzará después de que ocurra el evento \( e_i \). Este valor refleja las recompensas futuras esperadas si el agente llega a ese estado.
- \( $\sum_{i=1}^{m}$ \): Es una **suma sobre los eventos posibles** \( e_i \). Aquí \( m \) es el número total de eventos posibles que pueden ocurrir después de tomar la acción \( a \) en el estado \( s \). La suma acumula las recompensas y valores esperados de todos los posibles eventos, ponderados por las probabilidades de que cada uno de esos eventos ocurra.

### Función de Valor

$$ V^*(s) = \max(Q_{s_a}) $$

- \( V(s) \): Es el valor del estado \( s \). Representa la recompensa total esperada que el agente puede obtener a partir de ese estado, siguiendo la política óptima.
- \( $Q_{s_a}$ \): Es el valor de tomar la acción \( a \) en el estado \( s \). Este valor incluye la recompensa inmediata de tomar la acción \( a \) en \( s \), más las recompensas futuras derivadas de la transición al siguiente estado.
- \( $\max(Q_{s_a})$ \): El valor de un estado \( s \) es igual al valor máximo entre todas las acciones posibles \( a \). Es decir, se toma la acción que produce la mayor recompensa total esperada descrita como \( V^*(s) \).


### Condiciones de Contorno

$$
V_s = 
\begin{cases}
s_5 & \text{si } (s_2 = \text{JPY} \wedge s_1 = 4), \\
s_6 & \text{si } (s_2 = \text{CHF} \wedge s_1 = 4).
\end{cases}
$$

Esta función indica que al finalizar el día 3, es decir, al iniciar el cuarto día, el valor esperado para ese estado final será el precio de la última moneda que eligió el agente.

A partir de todo el planteamiento anterior, se realizó el siguiente código que refleja el mismo modelo pero adaptado para la librería rp4Solver. Al ejecutar la siguiente celda, se descargará o reescribirá el archivo en formato `py` que contiene la estructura SATR del modelo.

## Implementación en Python

In [7]:
%%writefile m_Caso_Final.py
'''
Model: MDP for trading optimal policy
Objective: Profit Maximization
'''

import pandas as pd #Para manejo de datos en DataFrames y funciónes de analítica de datos
import numpy as np #Para manejo de datos numéricos y arreglos
import matplotlib.pyplot as plt #Para la visualización de datos
import seaborn as sns #Para la visualización de datos
import scipy.stats as stats #Para realizar pruebas de hipótesis y estadiísticas
import warnings #Para evitar promptes de advertencia
warnings.filterwarnings('ignore') # evita warnings innecesarios al ejecutar código
import itertools

def load_data(url):
    df = pd.read_csv(url)
    df.columns = ['Fecha', ' Año', 'Nombre_Moneda', 'Continente', 'Cambio', 'Tipo_tasa', 'TRM', 'Id_Moneda']
    weekends = df['Fecha'].unique().tolist()[2::7]+df['Fecha'].unique().tolist()[3::7]
    keep = set(df['Fecha'].unique().tolist())
    dates = keep.difference(weekends)
    dates =list(dates)
    dates.sort()
    df = df[df['Fecha'].isin(dates)]
    COP_Med_Data = df.query("Cambio == 'Pesos colombianos por cada moneda' and Tipo_tasa == 'Media'")
    COP_Med_Data['TRM'] = COP_Med_Data['TRM'].str.replace(',','.').astype(float)
    data_original = pd.DataFrame(columns=df['Id_Moneda'].unique().tolist(), index=df['Fecha'].unique())
    for i in data_original.index:
        row = list(COP_Med_Data.query("Fecha == @i")['TRM'])
        data_original.loc[i] = row
    data_original = data_original.astype(float)
    data_original.head()
    data = data_original.copy()
    data.reset_index(drop=True, inplace=True)
    return data

link_database = 'https://raw.githubusercontent.com/NicolasCacer/bases-de-datos/main/1.2.TCM_Serie%20para%20un%20rango%20de%20fechas%20dado.csv'

data = load_data(url=link_database)

Estados = ['B','S']

def final_states(D=data, currency="JPY", n_per=20, only_states=True, rolling_std=False):  # la ventana de tiempo para la desviación móvil es 20 días
    currency = currency.upper()
    D = pd.DataFrame(data[currency])
    D['Mob_std'] = D[currency].rolling(n_per).std()
    D['Cambio_%'] = D[currency].pct_change()
    if rolling_std:
        D['sigma'] = D['Cambio_%'].rolling(n_per).std()
    D = D[n_per:]
    Estados = ['B','S']
    D['Estado'] = np.where(D['Cambio_%'] < 0, Estados[0], Estados[1])
    if only_states:
        D = D.drop(columns=[currency, 'Mob_std', 'Cambio_%'])
    D.reset_index(drop=True, inplace=True)
    return D

def markoviana(currency = "JPY", n_periods = 20, prints=True):

    Data = final_states(currency= currency, n_per= n_periods)
    Data.columns = ["x_t"]
    Data["x_t+1"] = Data["x_t"].shift(-1)
    Data["x_t+2"] = Data["x_t+1"].shift(-1)
    freq_obs = pd.DataFrame(columns=("Anterior", "Actual"))
    for i in Estados:
        for j in Estados:
            freq_obs.loc[len(freq_obs)] = [i,j]
    freq_obs = pd.concat([freq_obs,pd.DataFrame(np.zeros((freq_obs.shape[0],len(Estados)+1)),columns=Estados+['Total'])], axis=1)
    for i in range(len(Data)-2):
        secuencia = Data.loc[i].tolist()
        freq_obs.loc[(freq_obs['Anterior'] == secuencia[0]) & (freq_obs['Actual'] == secuencia[1]),secuencia[2]] += 1
    freq_obs['Total'] = freq_obs[Estados].sum(axis=1)
    freq_esp = freq_obs.copy()
    freq_esp.iloc[:,2:] = 0
    for i in range(len(Data)-2):
        secuencia = Data.loc[i].tolist()
        freq_esp.loc[freq_esp['Actual'] == secuencia[1],secuencia[2]] += 1
    freq_esp['Total'] = freq_esp[Estados].sum(axis=1)
    porcentaje_menor_5 = (freq_obs[Estados] < 5).mean() * 100
    hay_ceros = (freq_obs["Total"] == 0).any()
    porcentaje_menor_5 = (freq_obs[Estados] < 5).mean() * 100
    hay_ceros = (freq_obs["Total"] == 0).any()
    if (porcentaje_menor_5.mean() > 25 or hay_ceros) and prints:
        print("Se necesito agrupar dado que los datos menores a 5 represetan más del 25% del total o hay 0 en la columna total")
        print("---"*100)
        freq_obs = freq_obs.groupby("Anterior").sum().reset_index()
        freq_esp = freq_esp.groupby("Anterior").sum().reset_index()
    prob_obs = freq_obs[Estados].div(freq_obs["Total"], axis=0)
    prob_esp = freq_esp[Estados].div(freq_esp["Total"], axis=0)
    resultado = ((prob_obs - prob_esp) ** 2 / prob_esp).sum(axis=1) * freq_obs["Total"]
    chi_2 = (((prob_obs - prob_esp) ** 2 / prob_esp).sum(axis=1) * freq_obs["Total"]).sum()
    deg_free = (prob_esp.shape[1]-1)*(prob_esp.shape[0]-1)
    alpha = 0.05
    chi2_inv = stats.chi2.ppf(1-alpha, deg_free)
    if chi_2 < chi2_inv:
        Markov = "No se rechaza Ho"
    else:
        Markov = "Se rechaza Ho"
    freq_Markov = pd.DataFrame(0,columns= Estados+["Total"], index=Estados)
    for i in range(len(Data)-1):
        secuencia = Data.loc[i].tolist()[:2]
        freq_Markov.loc[secuencia[0],secuencia[1]]+= 1
    freq_Markov["Total"] = freq_Markov.sum(axis=1)
    prob_Markov = freq_Markov[Estados].div(freq_Markov["Total"], axis=0)
    if prints:
        print(f"-> Hipotesis markoviana\n {Markov}")
        print("-----"*10)
        print(f"-> Matriz de transición")
        print(prob_Markov.round(4))    
    return prob_Markov

################################## Condiciones de modelo ###################################################

divisas = ['JPY','CHF'] # Cuales divisas se va a utilizar en el aprendizaje supervisado
comission = 0.01 # Es la comisión de cambiar de divisa
matrices = {divisas[i]:markoviana(divisas[i],prints=False) for i in range(len(divisas))} # Es un diccionario el cual contiene la matriz de trasición para cada divisa
sigma_1 = np.round(final_states(currency=divisas[0], only_states=True, rolling_std=True)['sigma'].tolist()[-1], 4) # Es la ultima desviación movil de la primera divisa
sigma_2 = np.round(final_states(currency=divisas[1], only_states=False, rolling_std=True)['sigma'].tolist()[-1], 4) # Es la ultima desviación movil de la segunda divisa
matrix = matrices[divisas[0]] # Es la matriz de transición de la primera divisa
p_matrix = pd.DataFrame(columns=matrix.stack().index, index=matrix.stack().index) # Son todos los posibles estados de conjuntos de S y B
for i in p_matrix.index:
    for j in p_matrix.columns:
        p_matrix.loc[i,j] = ((matrices[divisas[0]].loc[i[0],j[0]]*matrices[divisas[1]].loc[i[1],j[1]]).round(4)) # Es la multiplicación de componente por componente para las dos divisas, genera un resultado de una matriz de transición de 4*4 que los estadosson duplas

################################## Funciones de modelo ###################################################

def Starting_State():  
    s1 = 1 # Es el inicio del día antes que se abra el mercado 
    s2 = divisas[0] # Es la divisa en el primer dia que comienza
    s3 = 'Sube' # Es el estado que comienza en el primer dia para la primera divisa
    s4 = 'Sube' # Es el estado que comienza en el primer día para la segunda divisa
    s5 = 100 # Es la contidad incial que comienza en el primer dia con la primera divisa
    s6 = 100 # Es la cantidad incial que comienza en el primer día con la primera divisa
    s = (s1,s2,s3,s4,s5,s6)
    return s

def Action_Set(s):   
    return ['mantener divisa','cambiar divisa'] # Son las posibles acciones que puede tomar con los diferentes Estados 

def Event_Set(s,a):
    event_list = [[f'Sube {i}', f'Baja {i}'] for i in divisas] # Son las posibles combinaciones entre S y B teniendo en cuenta su divisa
    ES = list(itertools.product(*event_list)) # Es para que esas combinaciones se creen en una lista
    return ES    

def Transition_Equations(s,a,e):
    (s1, s2, s3, s4, s5, s6) = s
    sn1 = s1 + 1 # Es para que en el siguiente estado el dia avance una unidad
    if a == 'cambiar divisa':
        if s2 == divisas[0]:
            sn2 = divisas[1] 
        else:
            sn2 = divisas[0]
    elif a == 'mantener divisa':
        sn2 = s2 # Esto verifica si la acción es cambiar, si se cambia la moneda, se verifica que divisa estamos y sí se cambia a la otra. Sin embargo, si la acción es matener el siguiente estado para s2 sera en la moneda que este en este momento 

    if e == (f'Sube {divisas[0]}', f'Sube {divisas[1]}') or e == (f'Sube {divisas[0]}', f'Baja {divisas[1]}'):
        sn3 = 'Sube'
        sn5 = s5 * (1 + sigma_1)
    elif e == (f'Baja {divisas[0]}', f'Sube {divisas[1]}') or e == (f'Baja {divisas[0]}', f'Baja {divisas[1]}'):
        sn3 = 'Baja'
        sn5 = s5 * (1 - sigma_1)

    if e == (f'Baja {divisas[0]}', f'Sube {divisas[1]}') or e == (f'Sube {divisas[0]}', f'Sube {divisas[1]}'):
        sn4 = 'Sube'
        sn6 = s6 * (1 + sigma_2)
    elif e == (f'Baja {divisas[0]}', f'Baja {divisas[1]}') or e == (f'Sube {divisas[0]}', f'Baja {divisas[1]}'):
        sn4 = 'Baja'
        sn6 = s6 * (1 - sigma_2) # Verifica si el evento en el que esta en este momento  va a subir o va a bajar si para el evento es de bajada para S3 y S4 su siguiente estado va se B (Bajada), y s5 y s6 van a ser restado segun su desviación respectiva. En cambio si el evento siguiente es Sube S3 y S4 sus estados siguientes van a ser S (Subida), y S5 y s6 su precio va a aumentar 

    sn = (sn1, sn2, sn3, sn4, np.round(sn5,4), np.round(sn6,4))
    return sn

def Constraints(s,a,sn,L):
    (s1, s2, s3, s4, s5, s6) = s
    Ct = (s1 <= 3) # Aqui se tiene como restricción que el numero de dia al inicio del mercado no puede ser mayor a 3
    return Ct

def Transition_Probabilities(s,a,e):
    (s1, s2, s3, s4, s5, s6) = s
    return p_matrix.loc[(s3[0],s4[0]),tuple([event[0] for event in e])]
    
def Action_Contribution(s,a):
    (s1, s2, s3, s4, s5, s6) = s
    if a == 'cambiar divisa':
        if s2 == divisas[0]:
            ca = -comission * s5
        else:
            ca = -comission * s6
    else:
        ca = 0 # Se verifia si acción fue cambiar si la acción fue cambiar se verifica en que divisa estoy en este momento y se cobrara una comisión del 0.01 del precio de la divisa en ese momento. Si la acción es mantener no se cobrara ninguna comisión 
    return np.round(ca, 4)

def Event_Contribution(s,a,e):
    (s1, s2, s3, s4, s5, s6) = s
    if s2 == divisas[0]:
        if e == (f'Sube {divisas[0]}', f'Sube {divisas[1]}') or e == (f'Sube {divisas[0]}', f'Baja {divisas[1]}'):
            ce = s5 * sigma_1
        else:
            ce = -s5 * sigma_1
    else:
        if e == (f'Sube {divisas[0]}', f'Sube {divisas[1]}') or e == (f'Baja {divisas[0]}', f'Sube {divisas[1]}'):
            ce = s6 * sigma_2
        else:
            ce = -s6 * sigma_2 # La contribución del evente verifica cual es el evento y se suma o se resta el precio de la divisa por el sigma correspondiente dependiendo si sube o baja la divisa 
    return np.round(ce, 4)

def Quality_Function(m,p,ca,ce,V_sn):
    Q_s_a = ca + sum(p[i] * (ce[i] + V_sn[i]) for i in range(0, m))
    return np.round(Q_s_a, 4) # La función de la calidad tiene todos los estados S y todos las acciones A y calcula la utilidad dependiendo de la contribución de la acción, del evente , la probabilidad y su estado futuro 

def Optimal_Value_Function(Q_s_a):
    V_s = max(Q_s_a) # Con todas las acciones y estados se maxima para determinar cual es la mejor acción que se debe tomar para cada estado
    return V_s

def Boundary_Condition(s):    
    (s1, s2, s3, s4, s5, s6) = s
    if s1 == 4:
        if s2 == divisas[0]:
            V_s = np.round(s5, 4)
        else:
            V_s = np.round(s6, 4)
    else:
        V_s = 0 # Se verifica si se s1 llega al inicio del 4 dia antes que se abra los mercados, si se llega el dia todos los estados s1 4 su condición de contorno este determinado por el precio de la divisa ese dia. Si no se ha llegado al cuerto dia su condición de contorno sera de 0 
    return V_s

Overwriting m_Caso_Final.py


De esta manera, el rp4solver ya puede acceder al archivo `.py` de la aplicación. Puede ser necesario reiniciar la sesión para que se pueda cargar el archivo generado adecuadamente en caso de error.

In [8]:
problema=atico("m_Caso_Final")

Model m_Caso_Final was imported


Ya con el problema cargado en la variable de `problema`, es posible generar los estados y generar el proceso de solución del modelo.

In [9]:
problema.State_Generation() # Se genera todos los possibles estados de la lista s
problema.report_detail = False
v,a = problema.Solution_Process('VAI',3) # Se crea la solución retornando dos vectores, el primer vector indica el el valor óptimo de ese estado, y el segundo vector es la mejor acción para maximizar la utilidad de la divisa 

States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 100, 100)                V(s)= 101.5008     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.24, 101.04)          V(s)= 102.118      π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.24, 98.96)           V(s)= 101.8455     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.76, 101.04)           V(s)= 101.0897     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.76, 98.96)            V(s)= 99.9854      π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.24, 101.04)          V(s)= 102.144      π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.24, 98.96)           V(s)= 101.3677     π(s)=mantener divisa
s= (2, 'CHF', 'Baja', 'Sube', 98.76, 101.04)      

*  El modelo importado, `m_Caso_Final`, ha generado 113 estados posibles.

* La solución se calcula con un horizonte infinito, lo que implica un proceso de optimización que busca maximizar la recompensa a largo plazo sin un límite de tiempo definido.

* La función de valor convergió en el paso `t=4`, indicando que en este punto las actualizaciones de la función de valor se estabilizaron.

* La función de valor óptima `V(s)`, que representa la expectativa de recompensa a partir de ese estado al seguir la política óptima.

* La acción óptima `π(s)`, que es la acción que maximiza el valor esperado en ese estado.

## Ganancia óptima
Teniendo en cuenta el resultado del algoritmo, se puede interpretar la función de valor para conocer cual es la ganancia óptima.

In [10]:
import contextlib
import io
def ganancia_óptima(modelo=problema, horizonte=3, precio=100):
    with contextlib.redirect_stdout(io.StringIO()): # Suprimir las impresiones
        v, a = modelo.Solution_Process('VAI', horizonte)
    max_gain = (v[0] - precio) / precio * 100
    print(f'La ganancia óptima es de {max_gain:.4f}%')
    return max_gain
gain_base = ganancia_óptima(problema)

La ganancia óptima es de 1.5008%


Es esta parte se evidencia el paso a paso de como esta tomando la desición la maquina para elegir el mejor valor V(sn) el cual maximice la utilidad ganada y con esa información toma la politica de si se cambia o no se cambia de divisa. En esta función de valor es de interes el primer valor del vector debido a que el modelo va en retroceso, implicando que el primer valor pertenece al último día.

# Análisis de sensibilidad

### Usando las divisas GBP - MXN

In [11]:
as1=atico("m_SensGBP_MXN")
as1.State_Generation() # Se genera todos los possibles estados de la lista s
as1.report_detail = False
v,a = as1.Solution_Process('VAI',3)
gain_as1 = ganancia_óptima(as1)

Model m_SensGBP_MXN was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'GBP', 'Sube', 'Sube', 100, 100)                V(s)= 101.4702     π(s)=mantener divisa
s= (2, 'GBP', 'Sube', 'Sube', 100.82, 101.38)          V(s)= 101.8843     π(s)=mantener divisa
s= (2, 'GBP', 'Sube', 'Baja', 100.82, 98.62)           V(s)= 101.3926     π(s)=mantener divisa
s= (2, 'GBP', 'Baja', 'Sube', 99.18, 101.38)           V(s)= 101.5584     π(s)=cambiar divisa
s= (2, 'GBP', 'Baja', 'Baja', 99.18, 98.62)            V(s)= 100.1621     π(s)=mantener divisa
s= (2, 'MXN', 'Sube', 'Sube', 100.82, 101.38)          V(s)= 102.534      π(s)=mantener divisa
s= (2, 'MXN', 'Sube', 'Baja', 100.82, 98.62)           V(s)= 101.2464     π(s)=mantener divisa
s= (2, 'MXN', 'Baj

Con las divisas GBP y MXN, la ganancia óptima fue del 1.4702%, siendo menor a 1.8260% del caso base con JPY y CHF. Esto puede deberse a diferencias en la volatilidad y condiciones iniciales de las divisas, que impactan las probabilidades de transición y las decisiones óptimas del modelo.

### Iniciando con la Divisa CHF

In [12]:
as11=atico("m_SensCHF_JPY")
as11.State_Generation() # Se genera todos los possibles estados de la lista s
as11.report_detail = False
v,a = as11.Solution_Process('VAI',3)
gain_as11 = ganancia_óptima(as11)

Model m_SensCHF_JPY was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'CHF', 'Sube', 'Sube', 100, 100)                V(s)= 101.7851     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.08, 101.34)          V(s)= 102.2611     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.08, 98.66)           V(s)= 101.9158     π(s)=mantener divisa
s= (2, 'CHF', 'Baja', 'Sube', 98.92, 101.34)           V(s)= 101.5091     π(s)=mantener divisa
s= (2, 'CHF', 'Baja', 'Baja', 98.92, 98.66)            V(s)= 100.2508     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.08, 101.34)          V(s)= 102.2848     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.08, 98.66)           V(s)= 101.1787     π(s)=mantener divisa
s= (2, 'JPY', 'Ba

Con CHF como divisa inicial, la ganancia óptima fue del 1.7851%, ligeramente inferior al 1.8260% del caso base con JPY. Esto puede atribuirse a las diferencias en las tendencias iniciales y probabilidades de transición asociadas al CHF.
### Cambiando los Porcentajes de Comisión

In [13]:
as2=atico("m_SensComB")
as2.State_Generation() # Se genera todos los possibles estados de la lista s
as2.report_detail = False
v,a = as2.Solution_Process('VAI',3)
gain_as2 = ganancia_óptima(as2)

Model m_SensComB was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 100, 100)                V(s)= 101.8864     π(s)=cambiar divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.24, 101.04)          V(s)= 102.3712     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.24, 98.96)           V(s)= 101.8455     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.76, 101.04)           V(s)= 101.895      π(s)=cambiar divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.76, 98.96)            V(s)= 100.3063     π(s)=cambiar divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.24, 101.04)          V(s)= 102.4613     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.24, 98.96)           V(s)= 102.0412     π(s)=mantener divisa
s= (2, 'CHF', 'Baja', '

In [14]:
as3=atico("m_SensComS")
as3.State_Generation() # Se genera todos los possibles estados de la lista s
as3.report_detail = False
v,a = as3.Solution_Process('VAI',3)
gain_as3 = ganancia_óptima(as3)

Model m_SensComS was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 100, 100)                V(s)= 100.9687     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.24, 101.04)          V(s)= 101.855      π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.24, 98.96)           V(s)= 101.8455     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.76, 101.04)           V(s)= 99.6355      π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.76, 98.96)            V(s)= 99.6262      π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.24, 101.04)          V(s)= 101.8451     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.24, 98.96)           V(s)= 100.0563     π(s)=mantener divisa
s= (2, 'CHF', 'Baja'

Con una comisión del 0.5%, la ganancia óptima aumentó a 1.8864%, ya que los costos bajos permiten aprovechar más cambios estratégicos de divisas. Por otro lado, al subir la comisión al 10%, la ganancia se redujo drásticamente a 0.9687%, reflejando cómo costos elevados limitan las oportunidades de maximizar el rendimiento del modelo.

### Variables s3 y s4 inician en Baja

In [22]:
as4=atico("m_SensS3S4B")
as4.State_Generation() # Se genera todos los possibles estados de la lista s
as4.report_detail = False
v,a = as4.Solution_Process('VAI',3)
gain_as4 = ganancia_óptima(as4)

Model m_SensS3S4B was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Baja', 'Baja', 100, 100)                V(s)= 101.9151     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2848     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.9943     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.66, 101.08)           V(s)= 101.1787     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.66, 98.92)            V(s)= 100.0017     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2611     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.5091     π(s)=mantener divisa
s= (2, 'CHF', 'Baja

En este escenario, las variables s3(estado del JPY) y s4 (estado del CHF) comienzan con una tendencia inicial de baja, lo cual impacta las decisiones del modelo y, en consecuencia, la ganancia óptima obtenida.

### Variable s3 inicia en Baja y s4 inicia en Sube

In [16]:
as5=atico("m_SensS3bS4s")
as5.State_Generation() # Se genera todos los possibles estados de la lista s
as5.report_detail = False
v,a = as5.Solution_Process('VAI',3)
gain_as5 = ganancia_óptima(as5)

Model m_SensS3bS4s was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Baja', 'Sube', 100, 100)                V(s)= 101.8719     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2848     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.9943     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.66, 101.08)           V(s)= 101.1787     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.66, 98.92)            V(s)= 100.0017     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2611     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.5091     π(s)=mantener divisa
s= (2, 'CHF', 'Baj



### Variable s3 inicia en Sube y s4 inicia en Baja

In [17]:
as6=atico("m_SensS3sS4b")
as6.State_Generation() # Se genera todos los possibles estados de la lista s
as6.report_detail = False
v,a = as6.Solution_Process('VAI',3)
gain_as6 = ganancia_óptima(as6)

Model m_SensS3sS4b was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Baja', 100, 100)                V(s)= 101.6711     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2848     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.9943     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.66, 101.08)           V(s)= 101.1787     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.66, 98.92)            V(s)= 100.0017     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2611     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.5091     π(s)=mantener divisa
s= (2, 'CHF', 'Baj

Un inicio mixto en las tendencias de las divisas resalta la sensibilidad del modelo a las condiciones iniciales y a la correlación entre los estados de las variables. Si bien el modelo puede adaptarse parcialmente, las ganancias óptimas se ven limitadas en comparación con escenarios completamente favorables.

### Disminuyendo la Cantidad Inicial

In [18]:
as7=atico("m_SensS5S6B")
as7.State_Generation() # Se genera todos los possibles estados de la lista s
as7.report_detail = False
v,a = as7.Solution_Process('VAI',3)
gain_as7 =ganancia_óptima(as7, precio=10)

Model m_SensS5S6B was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 10, 10)                V(s)= 10.1624     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 10.134, 10.108)        V(s)= 10.2285     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 10.134, 9.892)         V(s)= 10.1994     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 9.866, 10.108)         V(s)= 10.1179     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 9.866, 9.892)          V(s)= 10.0002     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 10.134, 10.108)        V(s)= 10.2261     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 10.134, 9.892)         V(s)= 10.1508     π(s)=mantener divisa
s= (2, 'CHF', 'Baja', 'Sube', 9.866, 10.

### Aumentando la Cantidad Inicial

In [19]:
as8=atico("m_SensS5S6S")
as8.State_Generation() # Se genera todos los possibles estados de la lista s
as8.report_detail = False
v,a = as8.Solution_Process('VAI',3)
gain_as8 = ganancia_óptima(as8, precio=1000)

Model m_SensS5S6S was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 1000, 1000)                V(s)= 1016.2343     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 1013.4, 1010.8)            V(s)= 1022.8479     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 1013.4, 989.2)             V(s)= 1019.9427     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 986.6, 1010.8)             V(s)= 1011.787      π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 986.6, 989.2)              V(s)= 1000.0168     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 1013.4, 1010.8)            V(s)= 1022.6105     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 1013.4, 989.2)             V(s)= 1015.0905     π(s)=mantener divis

En el modelo original, la ganancia óptima entre las divisas CHF y JPY se calcula como 1.8260%. Se observa que, al modificar la cantidad inicial invertida, este valor no varía. Esto demuestra que la política óptima es robusta y las decisiones no dependen de la escala de la inversión inicial, sino de las relaciones de transición y las probabilidades entre los estados.

### Disminuyendo la Ventana de Tiempo para la Desviación Móvil 

In [20]:
as9=atico("m_SensVentB")
as9.State_Generation() # Se genera todos los possibles estados de la lista s
as9.report_detail = False
v,a = as9.Solution_Process('VAI',3)
gain_as9 = ganancia_óptima(as9)

Model m_SensVentB was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 100, 100)                V(s)= 100.7364     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 100.71, 100.61)          V(s)= 101.1126     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 100.71, 99.39)           V(s)= 101.0581     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 99.29, 100.61)           V(s)= 100.3328     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 99.29, 99.39)            V(s)= 99.888       π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 100.71, 100.61)          V(s)= 101.1565     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 100.71, 99.39)           V(s)= 100.4952     π(s)=mantener divisa
s= (2, 'CHF', 'Baja

### Aumentando la Ventana de Tiempo para la Desviación Móvil 

In [21]:
as10=atico("m_SensVentS")
as10.State_Generation() # Se genera todos los possibles estados de la lista s
as10.report_detail = False
v,a = as10.Solution_Process('VAI',3)
gain_as10 = ganancia_óptima(as10)

Model m_SensVentS was imported
States generated: 113
Horizon: Finite
Solved by Value Iteration, All States Based Calculation
States generated: 113
Calculating Solution ...
Number of t-steps computed by the solution: 3

REPORT: OPTIMAL VALUE FUNCTION AND POLICY
Notation: s=state, V=value function, π=policy's action
s= (1, 'JPY', 'Sube', 'Sube', 100, 100)                V(s)= 101.6234     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2848     π(s)=mantener divisa
s= (2, 'JPY', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.9943     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Sube', 98.66, 101.08)           V(s)= 101.1787     π(s)=mantener divisa
s= (2, 'JPY', 'Baja', 'Baja', 98.66, 98.92)            V(s)= 100.0017     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Sube', 101.34, 101.08)          V(s)= 102.2611     π(s)=mantener divisa
s= (2, 'CHF', 'Sube', 'Baja', 101.34, 98.92)           V(s)= 101.5091     π(s)=mantener divisa
s= (2, 'CHF', 'Baja

Al modificar la ventana de tiempo para calcular la desviación estándar móvil, la ganancia óptima cambia. Al disminuir la ventana en 10 unidades, la ganancia óptima es de 0.7364%, mientras que al aumentarla en 10 unidades, esta sube a 1.6234%. Esto indica que la sensibilidad del modelo a la volatilidad influye directamente en los rendimientos, reflejando que una ventana más corta incrementa la reacción a cambios bruscos y una más larga suaviza las fluctuaciones.

# Modelo adicional (Bono)

In [22]:
%%writefile m_Caso_Final_B.py
'''
Model: All or nothing (2 state variables)
Objective: Profit Maximization

'''

import pandas as pd #Para manejo de datos en DataFrames y funciónes de analítica de datos
import numpy as np #Para manejo de datos numéricos y arreglos
import matplotlib.pyplot as plt #Para la visualización de datos
import seaborn as sns #Para la visualización de datos
import scipy.stats as stats #Para realizar pruebas de hipótesis y estadiísticas
import warnings #Para evitar promptes de advertencia
warnings.filterwarnings('ignore') # evita warnings innecesarios al ejecutar código
import itertools

def load_data(url):
    df = pd.read_csv(url)
    df.columns = ['Fecha', ' Año', 'Nombre_Moneda', 'Continente', 'Cambio', 'Tipo_tasa', 'TRM', 'Id_Moneda']
    weekends = df['Fecha'].unique().tolist()[2::7]+df['Fecha'].unique().tolist()[3::7]
    keep = set(df['Fecha'].unique().tolist())
    dates = keep.difference(weekends)
    dates =list(dates)
    dates.sort()
    df = df[df['Fecha'].isin(dates)]
    COP_Med_Data = df.query("Cambio == 'Pesos colombianos por cada moneda' and Tipo_tasa == 'Media'")
    COP_Med_Data['TRM'] = COP_Med_Data['TRM'].str.replace(',','.').astype(float)
    data_original = pd.DataFrame(columns=df['Id_Moneda'].unique().tolist(), index=df['Fecha'].unique())
    for i in data_original.index:
        row = list(COP_Med_Data.query("Fecha == @i")['TRM'])
        data_original.loc[i] = row
    data_original = data_original.astype(float)
    data_original.head()
    data = data_original.copy()
    data.reset_index(drop=True, inplace=True)
    return data

link_database = 'https://raw.githubusercontent.com/NicolasCacer/bases-de-datos/main/1.2.TCM_Serie%20para%20un%20rango%20de%20fechas%20dado.csv'
data = load_data(url=link_database)

Estados = ['B3','B2','B1','S1','S2','S3']
def final_states(D=data, currency="JPY", n_per=20,only_states=True, rolling_std = False):  # la ventana de tiempo para la desviación móvil es 20 días
    currency = currency.upper()
    D = pd.DataFrame(data[currency])
    D['Mob_std'] = D[currency].rolling(n_per).std()
    D['Cambio_%'] = D[currency].pct_change()
    if rolling_std:
        D['sigma'] = D['Cambio_%'].rolling(n_per).std()
    D = D[n_per:]
    D['Estado'] = np.where(D['Cambio_%'] < -2 * D['Mob_std'] / D[currency], Estados[0],
                    np.where(((-2 * D['Mob_std'] / D[currency] <= D['Cambio_%']) & (D['Cambio_%'] < -1 * D['Mob_std'] / D[currency])), Estados[1],
                    np.where(((-1 * D['Mob_std'] / D[currency] <= D['Cambio_%']) & (D['Cambio_%'] < 0)), Estados[2],
                    np.where(((0 <= D['Cambio_%']) & (D['Cambio_%'] < D['Mob_std'] / D[currency])), Estados[3],
                    np.where(((D['Mob_std'] / D[currency] <= D['Cambio_%']) & (D['Cambio_%'] < 2 * D['Mob_std'] / D[currency])), Estados[4],Estados[5])))))
    if only_states:
        D = D.drop(columns=[currency, 'Mob_std', 'Cambio_%'])
    D.reset_index(drop=True, inplace=True)
    return D

def markoviana(currency = "JPY", n_periods = 20, prints=True):

    Data = final_states(currency= currency, n_per= n_periods)
    Data.columns = ["x_t"]
    Data["x_t+1"] = Data["x_t"].shift(-1)
    Data["x_t+2"] = Data["x_t+1"].shift(-1)
    freq_obs = pd.DataFrame(columns=("Anterior", "Actual"))
    for i in Estados:
        for j in Estados:
            freq_obs.loc[len(freq_obs)] = [i,j]
    freq_obs = pd.concat([freq_obs,pd.DataFrame(np.zeros((freq_obs.shape[0],len(Estados)+1)),columns=Estados+['Total'])], axis=1)
    for i in range(len(Data)-2):
        secuencia = Data.loc[i].tolist()
        freq_obs.loc[(freq_obs['Anterior'] == secuencia[0]) & (freq_obs['Actual'] == secuencia[1]),secuencia[2]] += 1
    freq_obs['Total'] = freq_obs[Estados].sum(axis=1)
    freq_esp = freq_obs.copy()
    freq_esp.iloc[:,2:] = 0
    for i in range(len(Data)-2):
        secuencia = Data.loc[i].tolist()
        freq_esp.loc[freq_esp['Actual'] == secuencia[1],secuencia[2]] += 1
    freq_esp['Total'] = freq_esp[Estados].sum(axis=1)
    porcentaje_menor_5 = (freq_obs[Estados] < 5).mean() * 100
    hay_ceros = (freq_obs["Total"] == 0).any()
    porcentaje_menor_5 = (freq_obs[Estados] < 5).mean() * 100
    hay_ceros = (freq_obs["Total"] == 0).any()
    if (porcentaje_menor_5.mean() > 25 or hay_ceros) and prints:
        print("Se necesito agrupar dado que los datos menores a 5 represetan más del 25% del total o hay 0 en la columna total")
        print("---"*100)
        freq_obs = freq_obs.groupby("Anterior").sum().reset_index()
        freq_esp = freq_esp.groupby("Anterior").sum().reset_index()
    prob_obs = freq_obs[Estados].div(freq_obs["Total"], axis=0)
    prob_esp = freq_esp[Estados].div(freq_esp["Total"], axis=0)
    chi_2 = (((prob_obs - prob_esp) ** 2 / prob_esp).sum(axis=1) * freq_obs["Total"]).sum()
    deg_free = (prob_esp.shape[1]-1)*(prob_esp.shape[0]-1)
    alpha = 0.05
    chi2_inv = stats.chi2.ppf(1-alpha, deg_free)
    if chi_2 < chi2_inv:
        Markov = "No se rechaza Ho"
    else:
        Markov = "Se rechaza Ho"
    freq_Markov = pd.DataFrame(0,columns= Estados+["Total"], index=Estados)
    for i in range(len(Data)-1):
        secuencia = Data.loc[i].tolist()[:2]
        freq_Markov.loc[secuencia[0],secuencia[1]]+= 1
    freq_Markov["Total"] = freq_Markov.sum(axis=1)
    prob_Markov = freq_Markov[Estados].div(freq_Markov["Total"], axis=0)
    if prints:
        print(f"-> Hipotesis markoviana\n {Markov}")
        print("-----"*10)
        print(f"-> Matriz de transición")
        print(prob_Markov.round(4))
    return prob_Markov

#################################### Condiciones del Modelo #################################################

divisas = ['JPY','CHF']  
comission = 0.01
sigmas = {}
for divisa in divisas:
    sigma_value = np.round(final_states(currency=divisa, only_states=True, rolling_std=True)['sigma'].tolist()[-1], 4)
    sigmas[divisa] = sigma_value 
Estados = ['S1', 'S2', 'S3', 'B1', 'B2', 'B3']  
matrices = {divisa: markoviana(divisa, prints=False) for divisa in divisas}
resultado_kron = np.array([[1]])
for key, matriz in matrices.items():
    resultado_kron = np.kron(resultado_kron, matriz.values)
indices = pd.MultiIndex.from_product([matriz.index for matriz in matrices.values()],
                                     names=[f'Matriz{i+1}' for i in range(len(matrices))])
nombres_indices = ["".join(tupla) for tupla in indices]
markov_F = pd.DataFrame(resultado_kron , columns=indices, index=indices)

#################################### Estructuración del Modelo #################################################    

def Starting_State():
    s1 = 1  
    s2 = divisas[0]  
    divisa_states = {divisa: 'S1' for divisa in divisas}  
    s_vals = {divisa: 100 for divisa in divisas}  
    return s1,s2, divisa_states , s_vals

def Action_Set(s):
    return ['mantener divisa', 'cambiar divisa']

def Event_Set(s,a):
    return indices

def Transition_Equations(s, a, e):
    s1, s2, divisa_states , s_vals = s
    sn1 = s1 + 1  # Incrementar el paso
    sn_divisa_states = divisa_states.copy()
    sn_vals = s_vals.copy()
    for i in range(len(divisas)): # Asignar el estado futuro de la moneda segun cada evento
        for estados in Estados:
            posibles =  [tupla for tupla in  indices if tupla[i] == estados]
            if e in posibles:
                sn_divisa_states[divisas[i]] = e[i]
                signo = 1 if e[i][0] == 'S' else -1
                sn_vals[divisas[i]] = np.round(s_vals[divisas[i]]* (signo * (0.5 + 1 * (int(e[i][1]) - 1)) * sigmas[divisas[i]] + 1),4)
    if a == 'cambiar divisa':
        current_index = divisas.index(s2)
        sn2 = divisas[(current_index + 1) % len(divisas)]
    else:
        sn2 = s2
    return sn1 , sn2 ,sn_divisa_states , sn_vals

def Constraints(s,a,sn,L):
    s1, *resto = s
    Ct = (s1 <= 3)
    return Ct

def Transition_Probabilities(s, a, e):
    s1,s2 ,divisa_states , s_vals = s
    divisa_states_values = list(divisa_states.values())
    return markov_F.loc[(tuple(divisa_states_values)),e]

def Action_Contribution(s, a):
    s1,s2, divisa_states, s_vals= s
    
    if a == 'cambiar divisa':
        return -comission * s_vals[s2]
    return 0

def Event_Contribution(s, a, e):
    s1, s2, divisa_states, s_vals = s
 
    for i in range(len(divisas)): # Asignar el estado futuro de la moneda segun cada evento
        if s2 == divisas[i]:
            posibles =  [tupla for tupla in indices if tupla[i][0] == 'S']
            if e in posibles:
                ce = sigmas[divisas[i]]*s_vals[divisas[i]]
            else:
                ce = -sigmas[divisas[i]]*s_vals[divisas[i]]
    return np.round(ce,4)

def Quality_Function(m, p, ca, ce, V_sn):
    return np.round(ca + sum(p[i] * (ce[i] + V_sn[i]) for i in range(m)), 4)

def Optimal_Value_Function(Q_s_a):
    return  max(Q_s_a)

def Boundary_Condition(s):
    s1, s2, divisa_states, s_vals= s
    if s1 == 4:  # Paso final
        return np.round(s_vals[s2], 4)
    return 0

Overwriting m_Caso_Final_B.py


A partir de esta versión del modelo es posible construir situaciones con 2, 3 o 4 de las divisas seleccionadas. Así mismo cuenta con 6 estados posibles para cada divisa. Por cuestiones de tiempos de computación, se decdió no usar más de dos divisas, además que al ser parecido al modelo base, sería posible realizar una comparación entre el efecto de ampliar el conjunto de estados para cada divisa.

In [23]:
Bono = atico("m_Caso_Final_B")
Bono.State_Generation() # Se genera todos los possibles estados de la lista s
Bono.report_detail = False
v,a = Bono.Solution_Process('VAI',3) # Se crea la solución retornando dos vectores, el primer vector indica el el valor óptimo de ese estado, y el segundo vector es la mejor acción para maximizar la utilidad de la divisa 

  V(s)= 104.3824     π(s)=myself
s= (4, 'JPY', {'JPY': 'S3', 'CHF': 'S2'}, {'JPY': 104.3824, 'CHF': 102.575})       V(s)= 104.3824     π(s)=myself
s= (4, 'JPY', {'JPY': 'S3', 'CHF': 'S3'}, {'JPY': 104.3824, 'CHF': 103.6254})      V(s)= 104.3824     π(s)=myself
s= (4, 'JPY', {'JPY': 'S3', 'CHF': 'B1'}, {'JPY': 104.3824, 'CHF': 100.4742})      V(s)= 104.3824     π(s)=myself
s= (4, 'JPY', {'JPY': 'S3', 'CHF': 'B2'}, {'JPY': 104.3824, 'CHF': 99.4238})       V(s)= 104.3824     π(s)=myself
s= (4, 'JPY', {'JPY': 'S3', 'CHF': 'B3'}, {'JPY': 104.3824, 'CHF': 98.3734})       V(s)= 104.3824     π(s)=myself
s= (4, 'JPY', {'JPY': 'B1', 'CHF': 'S1'}, {'JPY': 100.6161, 'CHF': 101.5246})      V(s)= 100.6161     π(s)=myself
s= (4, 'JPY', {'JPY': 'B1', 'CHF': 'S2'}, {'JPY': 100.6161, 'CHF': 102.575})       V(s)= 100.6161     π(s)=myself
s= (4, 'JPY', {'JPY': 'B1', 'CHF': 'S3'}, {'JPY': 100.6161, 'CHF': 103.6254})      V(s)= 100.6161     π(s)=myself
s= (4, 'JPY', {'JPY': 'B1', 'CHF': 'B1'}, {'JPY': 100.6

In [25]:
max_gain = (v[0] - 100) / 100 * 100
print(f'La ganancia óptima es de {max_gain:.4f}% cuando se inicia con el JPY, \n'
          f'tanto el JPY como CHF deben estar en alza pequeña (S1).')

La ganancia óptima es de 0.7792% cuando se inicia con el JPY, 
tanto el JPY como CHF deben estar en alza pequeña (S1).


Como se puede observar, la rentabilidad es inferior a la del modelo base. Esto podría explicarse por el hecho de que, al ser un modelo más completo y posiblemente más fiel a la realidad, refleja cómo la volatilidad de los mercados dificulta la obtención de rentabilidad al invertir en divisas. Sin embargo, sería útil explorar este modelo con mayor profundidad, teniendo en cuenta que es costoso desde el punto de vista computacional.

# Análisis y Conclusiones

A partir de todo lo realizado en este documento, se logro construir exitosamente el modelo base, el cual consiste del uso de las divisas Yen Japones y Franco Suizo, a través de lo cual se pudo determinar una política optima de inversión para un horizonte de planeación de 3 días; con lo cual, se estableció la ganancia optima esperada, la cual es de 1.5008% de rentabilidad en este lapso de tiempo. 

Con base en el análisis de sensibilidad, se observó que con las divisas MXN y GBP, se puede alcanzar una rentabilidad de hasta 1.4702%; de la misma manera, se pudo determinar que la comisión, las divisas y el estado inicial de cada moneda son los factores que tienen mayor impacto en el rendimiento del modelo.

Por último, se puede observar que para el modelo MDP con estados adicionales, se pudo lograr una rentabilidad de hasta 0.7792%, lo cual indica que se espera una mayor ganancia con el modelo base. 
En resumen a este documento, se utilizan modelos de deep learning, el cual funciona de manera no determinística, y por medio del modelamiento de cadenas de Markov del mercado bursátil de las diferentes  divisas, se pudo establecer una política de inversión para maximizar las ganancias esperadas en un entorno volátil como lo es el mercado de acciones. 

<h1 style="text-align: center;">Referencias</h1>

- Aronsson, M., & Folkesson, A. (2023). Stock market analysis with a Markovian approach: Properties and prediction of OMXS30. https://kth.diva-portal.org/smash/get/diva2:1823899/FULLTEXT01.pdf

- Barbu, S., Amico, G. & De Blasis, R. (2017). Novel advancements in the Markov chain stock model: analysis and inference. Ann Finance 13, 125–152. https://doi.org/10.1007/s10436-017-0297-9 

- Banrep. (2024). Billetes y Monedas - Monedas Disponibles. https://www.banrep.gov.co/es/estadisticas/monedas-disponibles

- Svoboda, M., & Říhová, P. (2021). Stock price prediction using markov chains analysis with varying state space on data from the czech republic. E+M Ekonomie a Management, 24, 142-155. https://doi.org/10.15240/tul/001/2021-4-009

- Rodríguez-González, A., García-Crespo, Á., Colomo-Palacios, R., Guldrís Iglesias, F., & Gómez-Berbís, J. M. (2011). CAST: Using neural networks to improve trading systems based on technical analysis by means of the RSI financial indicator. Expert Systems with Applications, 38 (9), 11489-11500. https://doi.org/https://doi.org/10.1016/j.eswa.2011.03.023

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=422d8a82-4e6c-4387-b307-483f4df65536' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>